## Data Loading and Initial Exploration

This block loads the three Stata datasets (`tender`, `bid`, and `bidder`) and performs basic consistency checks before analysis.

- Define the data directory and verify that all required `.dta` files are available.  
- Load each file and ensure identifiers such as `tender_id` and `year` are stored as integers rather than floats.  
- Display a quick preview of the `tender` dataset to confirm that locations and years were imported correctly.  
- Print structural information (shape, data types, and unique values) to confirm that the data is clean and ready for merging.



In [56]:
from IPython.display import display
from pathlib import Path
import pandas as pd

# tender
DATA_DIR = Path("E:/Daniel/Stockholm/submission/stata/data")
tender_path = DATA_DIR / "tender.dta"

# bid
bid_path = DATA_DIR / "bid.dta"
assert bid_path.exists(), f"Not found: {bid_path.resolve()}"

# bidder
bidder_path = DATA_DIR / "bidder.dta"
assert bidder_path.exists(), f"Not found: {bidder_path.resolve()}"


# Load data
tender = pd.read_stata(tender_path)
bid = pd.read_stata(bid_path)
bidder = pd.read_stata(bidder_path)

# Normalise integer-like columns
for col in ["tender_id", "year"]:
    if col in tender and tender[col].dtype.kind == "f":
        tender[col] = tender[col].astype("Int64") if tender[col].isna().any() else tender[col].astype(int)

# Quick view
display(tender.head(10))
print("Shape (rows, columns):", tender.shape)
print("Dtypes:\n", tender.dtypes)
print("Unique cities:", tender["location"].unique().tolist())
print("Years:", sorted(pd.Series(tender["year"]).dropna().unique().tolist()))


,tender_id,location,year
0,101,Budapest,2018
1,102,Budapest,2017
2,103,Budapest,2016
3,104,Budapest,2016
4,105,Budapest,2021
5,106,Miskolc,2018
6,107,Miskolc,2016
7,108,Debrecen,2017
8,109,Szeged,2019


Shape (rows, columns): (9, 3)
Dtypes:
 tender_id     int64
location     object
year          int64
dtype: object
Unique cities: ['Budapest', 'Miskolc', 'Debrecen', 'Szeged']
Years: [2016, 2017, 2018, 2019, 2021]


## Data Validation: Tender Identifiers and Locations

This step checks the integrity of the `tender` dataset before aggregation.

- **Duplicate check:**  
  Identify whether any `tender_id` values appear more than once.  
  If duplicates exist, display the affected rows to confirm whether they represent true duplicates or repeated entries.

- **Missing values check:**  
  Count and report any tenders with missing `location` information.




In [57]:
# Duplicate tender_id check
dup_mask = tender["tender_id"].duplicated(keep=False)
print("Duplicate tender_id rows:", int(dup_mask.sum()))
if dup_mask.any():
    display(tender[dup_mask].sort_values("tender_id"))

# Missing location check
missing_locs = tender["location"].isna().sum()
print("Rows with missing location:", int(missing_locs))


Duplicate tender_id rows: 0
Rows with missing location: 0


## Frequency of Tenders per City

This section summarises the spatial distribution of tenders across cities.

- Group the `tender` dataset by `location` and count the number of unique tenders associated with each city.  
- Rename the resulting variable as `n_tenders` and sort alphabetically by city for readability.  
- Perform a consistency check to verify that the sum of `n_tenders` matches the total number of tender records in the dataset.  
- Export the resulting summary table to `submission/stata/output/tenders_per_city.csv`.




In [58]:
# Count tenders per city
tenders_per_city = (
    tender.groupby("location", as_index=False)["tender_id"]
    .count()
    .rename(columns={"tender_id": "n_tenders"})
    .sort_values("location")
)

display(tenders_per_city)

# Consistency check
print("Sum of n_tenders:", int(tenders_per_city["n_tenders"].sum()), "Total tenders:", len(tender))

OUT_DIR = Path("submission/stata/output")
OUT_DIR.mkdir(parents=True, exist_ok=True)

out_csv = OUT_DIR / "tenders_per_city.csv"
tenders_per_city.to_csv(out_csv, index=False)
print("Saved:", out_csv.resolve())



,location,n_tenders
0,Budapest,5
1,Debrecen,1
2,Miskolc,2
3,Szeged,1


Sum of n_tenders: 9 Total tenders: 9
Saved: E:\Daniel\Stockholm\submission\notebooks\submission\stata\output\tenders_per_city.csv


## Data Normalisation: Bid Identifiers

Before analysing the bids, this block ensures that all identifier variables are correctly formatted.

- Convert `tender_id` and `bidder_id` from floating-point to integer values to avoid merge inconsistencies.  
- Display a sample of the `bid` dataset to confirm the structure and integrity of the data.  
- Print the dataset’s shape and count the number of unique tenders and bidders represented.



In [59]:
# Normalise IDs bid + tender
for col in ["tender_id", "bidder_id"]:
    if bid[col].dtype.kind == "f":
        bid[col] = bid[col].astype("Int64") if bid[col].isna().any() else bid[col].astype(int)

display(bid.head(10))
print("Shape:", bid.shape)
print("Unique tenders:", bid["tender_id"].nunique())
print("Unique bidders:", bid["bidder_id"].nunique())

,tender_id,bidder_id
0,101,1
1,103,1
2,101,2
3,101,7
4,101,10
5,101,12
6,102,2
7,102,11
8,103,8
9,103,10


Shape: (30, 2)
Unique tenders: 8
Unique bidders: 11


## Number of Bidders per Tender

This step measures the level of competition for each tender.

- Group the `bid` dataset by `tender_id` and count the number of distinct `bidder_id` values linked to each tender.  
- Store this result as a new variable, `n_bidders`, representing how many unique firms submitted bids for each project.  
- Display the first few rows to confirm the structure and check that the number of rows equals the total number of tenders present in the bid records.



In [60]:
# Count distinct bidders per tender
bidders_per_tender = (
    bid.groupby("tender_id", as_index=False)["bidder_id"]
    .nunique()
    .rename(columns={"bidder_id": "n_bidders"})
)

display(bidders_per_tender.head())
print("Total rows (tenders):", len(bidders_per_tender))

,tender_id,n_bidders
0,101,5
1,102,2
2,103,3
3,104,3
4,105,5


Total rows (tenders): 8


## Frequency Distribution of Bidders per Tender

This section summarises how many bidders each tender attracted.

- Use `value_counts()` to compute how many tenders had a given number of bidders (e.g., how many tenders had two bidders, three bidders, and so on).  
- Sort the results by the number of bidders (`n_bidders`) and rename the output column as `n_tenders`.  
- Verify data consistency by confirming that the total number of tenders in the distribution equals the number of tenders in the dataset.  
- Export the resulting frequency table to `submission/stata/output/bidders_per_tender_distribution.csv`.



In [61]:
# Frequency distribution of number of bidders
distribution = (
    bidders_per_tender["n_bidders"]
    .value_counts()
    .sort_index()
    .rename_axis("n_bidders")
    .reset_index(name="n_tenders")
)

display(distribution)

# Consistency check
print("Check:", int(distribution["n_tenders"].sum()), "==", len(bidders_per_tender))

out_csv = OUT_DIR / "bidders_per_tender_distribution.csv"
distribution.to_csv(out_csv, index=False)
print("Saved:", out_csv.resolve())

,n_bidders,n_tenders
0,2,1
1,3,3
2,4,1
3,5,3


Check: 8 == 8
Saved: E:\Daniel\Stockholm\submission\notebooks\submission\stata\output\bidders_per_tender_distribution.csv


## Data Preparation: Linking Bidders and Tenders

This section prepares the merged dataset required to analyse bidding patterns across cities.

- Standardise identifiers by converting all numeric IDs in `bidder`, `bid`, and `tender` datasets to integer format.  
- Merge `bid` with `bidder` to append the city of origin for each bidding company (`bidder_city`).  
- Merge the resulting dataset with `tender` to add the destination city of the project (`tender_city`).  
- Rename and reorganise columns to obtain a clean dataset that links each bid to both its origin and target locations.  
- Display a preview to verify that the resulting table includes the expected variables (`bidder_id`, `bidder_city`, `tender_id`, `tender_city`) and consistent dimensions.

Preparation for city pairs.


In [62]:
# Normalise integer-like columns
for col in ["bidder_id"]:
    if bidder[col].dtype.kind == "f":
        bidder[col] = bidder[col].astype(int)

display(bidder.head(10))
print("Shape bidder:", bidder.shape)
print("Unique bidder cities:", bidder["city"].unique().tolist())

# Normalise IDs on bid and tender
for df, cols in [(bid, ["tender_id", "bidder_id"]), (tender, ["tender_id", "year"])]:
    for c in cols:
        if df[c].dtype.kind == "f":
            df[c] = df[c].astype(int)

# Merge bid + bidder to add bidder_city
bids_with_bidder = bid.merge(bidder, on="bidder_id", how="left")

# Merge with tender to add tender_city
bids_enriched = bids_with_bidder.merge(
    tender[["tender_id", "location"]].rename(columns={"location": "tender_city"}),
    on="tender_id",
    how="left"
).rename(columns={"city": "bidder_city"})

display(bids_enriched.head(15))
print("Shape enriched:", bids_enriched.shape)
print(bids_enriched.columns.tolist())


,bidder_id,city
0,1,Budapest
1,2,Budapest
2,3,Miskolc
3,4,Debrecen
4,5,Budapest
5,6,Szeged
6,7,Budapest
7,8,Szeged
8,9,Miskolc
9,10,Budapest


Shape bidder: (12, 2)
Unique bidder cities: ['Budapest', 'Miskolc', 'Debrecen', 'Szeged']


,tender_id,bidder_id,bidder_city,tender_city
0,101,1,Budapest,Budapest
1,103,1,Budapest,Budapest
2,101,2,Budapest,Budapest
3,101,7,Budapest,Budapest
4,101,10,Budapest,Budapest
5,101,12,Budapest,Budapest
6,102,2,Budapest,Budapest
7,102,11,Budapest,Budapest
8,103,8,Szeged,Budapest
9,103,10,Budapest,Budapest


Shape enriched: (30, 4)
['tender_id', 'bidder_id', 'bidder_city', 'tender_city']


## Bid Matrix by City Pair

This section summarises bidding activity between all combinations of bidder and tender cities.

- Aggregate the enriched dataset to count the total number of bids for each `(bidder_city, tender_city)` pair.  
- Sort the results alphabetically to produce a clear tabular summary.  
- Validate the aggregation by confirming that the sum of all bids across city pairs equals the total number of bid records.  
- Export the summary table to `submission/stata/output/bids_per_city_pair.csv`.  
- Create a pivot table where:
  - Rows represent the bidder’s city of origin.  
  - Columns represent the tender’s city (destination).  
  - Cell values indicate the number of bids between each pair.

This matrix provides a direct visualisation of bidding flows across cities, capturing both local bids (same-city pairs) and cross-city competition.


In [63]:
city_pairs = (
    bids_enriched.groupby(["bidder_city", "tender_city"], as_index=False)["bidder_id"]
    .count()
    .rename(columns={"bidder_id": "n_bids"})
    .sort_values(["bidder_city", "tender_city"])
)

display(city_pairs)
print("Consistency check:", int(city_pairs["n_bids"].sum()), "==", len(bids_enriched))

out_csv = OUT_DIR / "bids_per_city_pair.csv"
city_pairs.to_csv(out_csv, index=False)
print("Saved:", out_csv.resolve())

pivot_city_pairs = city_pairs.pivot(index="bidder_city", columns="tender_city", values="n_bids").fillna(0).astype(int)
display(pivot_city_pairs)



,bidder_city,tender_city,n_bids
0,Budapest,Budapest,16
1,Budapest,Debrecen,2
2,Budapest,Miskolc,1
3,Debrecen,Debrecen,1
4,Debrecen,Miskolc,1
5,Miskolc,Debrecen,1
6,Miskolc,Miskolc,3
7,Szeged,Budapest,2
8,Szeged,Debrecen,1
9,Szeged,Miskolc,2


Consistency check: 30 == 30
Saved: E:\Daniel\Stockholm\submission\notebooks\submission\stata\output\bids_per_city_pair.csv


tender_city,Budapest,Debrecen,Miskolc
bidder_city,,,
Budapest,16,2,1
Debrecen,0,1,1
Miskolc,0,1,3
Szeged,2,1,2


## Constructing the Local Experience Framework (next 3 code blocks)

This section prepares the data required to calculate the `local_experience` variable.

- Append the tender year (`tender_year`) to the enriched bids dataset for temporal tracking.  
- Identify, for each bidder and tender city, the **first year** in which the bidder ever placed a bid there.  
  - This is achieved by grouping by `(bidder_id, tender_city)` and selecting the earliest observed `tender_year`.  
  - The resulting table (`first_year_by_city`) represents the start of each bidder’s local experience timeline.  
- Save the resulting table as an audit file (`first_year_in_city_by_bidder.csv`) for validation.  
- Build the **full bidder × tender universe**, representing every possible bidder–tender combination, including those where no actual bid occurred.  
  - This ensures that all potential bidding opportunities are evaluated when assigning `local_experience`.  





In [64]:
# Add tender_year into bids_enriched
tender_years = tender[["tender_id", "year"]].rename(columns={"year": "tender_year"})
bids_enriched = bids_enriched.merge(tender_years, on="tender_id", how="left")

display(
    bids_enriched[["bidder_id", "tender_id", "bidder_city", "tender_city", "tender_year"]]
    .head(10)
)

print("bids_enriched shape (rows, cols):", bids_enriched.shape)

,bidder_id,tender_id,bidder_city,tender_city,tender_year
0,1,101,Budapest,Budapest,2018
1,1,103,Budapest,Budapest,2016
2,2,101,Budapest,Budapest,2018
3,7,101,Budapest,Budapest,2018
4,10,101,Budapest,Budapest,2018
5,12,101,Budapest,Budapest,2018
6,2,102,Budapest,Budapest,2017
7,11,102,Budapest,Budapest,2017
8,8,103,Szeged,Budapest,2016
9,10,103,Budapest,Budapest,2016


bids_enriched shape (rows, cols): (30, 5)


In [65]:
# Compute first year the bidder ever bid in each tender city
first_year_by_city = (
    bids_enriched[["bidder_id", "tender_city", "tender_year"]]
    .drop_duplicates()
    .groupby(["bidder_id", "tender_city"], as_index=False)["tender_year"]
    .min()
    .rename(columns={"tender_year": "first_year_in_city"})
)

display(first_year_by_city.head(10))

# Save audit file (small and useful later)
first_year_csv = OUT_DIR / "first_year_in_city_by_bidder.csv"
first_year_by_city.to_csv(first_year_csv, index=False)

print(
    "first_year_by_city rows:",
    len(first_year_by_city),
    "| saved to:",
    first_year_csv.resolve()
)


,bidder_id,tender_city,first_year_in_city
0,1,Budapest,2016
1,2,Budapest,2016
2,3,Debrecen,2017
3,3,Miskolc,2016
4,4,Debrecen,2017
5,4,Miskolc,2018
6,5,Debrecen,2017
7,7,Budapest,2018
8,8,Budapest,2016
9,8,Debrecen,2017


first_year_by_city rows: 17 | saved to: E:\Daniel\Stockholm\submission\notebooks\submission\stata\output\first_year_in_city_by_bidder.csv


In [66]:
# Build the full bidder × tender universe
bidders_universe = bidder[["bidder_id"]].drop_duplicates()
tenders_universe = tender[["tender_id", "location", "year"]].rename(
    columns={"location": "tender_city", "year": "tender_year"}
)

# Use cross merge; fallback to key-merge if pandas version is older
try:
    universe = bidders_universe.merge(tenders_universe, how="cross")
except TypeError:
    bidders_universe = bidders_universe.assign(__key=1)
    tenders_universe = tenders_universe.assign(__key=1)
    universe = bidders_universe.merge(tenders_universe, on="__key").drop(columns="__key")

display(universe.head(10))

print("universe shape (rows, cols):", universe.shape)


,bidder_id,tender_id,tender_city,tender_year
0,1,101,Budapest,2018
1,1,102,Budapest,2017
2,1,103,Budapest,2016
3,1,104,Budapest,2016
4,1,105,Budapest,2021
5,1,106,Miskolc,2018
6,1,107,Miskolc,2016
7,1,108,Debrecen,2017
8,1,109,Szeged,2019
9,2,101,Budapest,2018


universe shape (rows, cols): (108, 4)


## Local Experience Variable and Yearly Distribution

This section creates the `local_experience` indicator for all possible bidder–tender combinations and summarises its distribution by year.

- Merge the `first_year_by_city` table into the full bidder–tender universe to attach each bidder’s earliest bidding year in the relevant city.  
- Ensure all key numeric columns (`first_year_in_city`, `tender_year`) are stored as integers.  
- Define the `local_experience` dummy variable:
  - Takes value `1` if the bidder has previously bid in the same tender city **before** the current tender’s year.  
  - Takes value `0` otherwise (including cases with no prior bids or missing data).  
- Inspect the resulting dataset to confirm correct logic and overall proportion of experienced bidders.  
- Produce a cross-tabulation (`crosstab`) showing the number of bidder–tender pairs with and without local experience by tender year.  
- Export the results to `submission/stata/output/local_experience_by_year.csv`.

This final output captures the temporal evolution of bidders’ local familiarity with tender locations, completing the analytical workflow.


In [67]:
# Merge first_year_in_city into the universe
universe = universe.merge(
    first_year_by_city, on=["bidder_id", "tender_city"], how="left"
)

# Ensure integer types
if "first_year_in_city" in universe.columns:
    universe["first_year_in_city"] = universe["first_year_in_city"].astype("Int64")
if "tender_year" in universe.columns:
    universe["tender_year"] = universe["tender_year"].astype("Int64")

# Create dummy for local experience
universe["local_experience"] = (
    (universe["first_year_in_city"] < universe["tender_year"]).astype("Int64")
).fillna(0).astype(int)

display(
    universe[["bidder_id", "tender_id", "tender_city", "tender_year", "first_year_in_city", "local_experience"]]
    .head(10)
)

print("share with local experience:", round(universe["local_experience"].mean(), 4))

universe_out = OUT_DIR / "universe_with_local_experience.csv"

cols_export = [
    "bidder_id",
    "tender_id",
    "tender_city",
    "tender_year",
    "first_year_in_city",
    "local_experience"
]

universe[cols_export].to_csv(universe_out, index=False)
print("Full universe saved to:", universe_out.resolve())
print("Rows exported:", len(universe))

,bidder_id,tender_id,tender_city,tender_year,first_year_in_city,local_experience
0,1,101,Budapest,2018,2016,1
1,1,102,Budapest,2017,2016,1
2,1,103,Budapest,2016,2016,0
3,1,104,Budapest,2016,2016,0
4,1,105,Budapest,2021,2016,1
5,1,106,Miskolc,2018,<NA>,0
6,1,107,Miskolc,2016,<NA>,0
7,1,108,Debrecen,2017,<NA>,0
8,1,109,Szeged,2019,<NA>,0
9,2,101,Budapest,2018,2016,1


share with local experience: 0.1852
Full universe saved to: E:\Daniel\Stockholm\submission\notebooks\submission\stata\output\universe_with_local_experience.csv
Rows exported: 108


In [68]:
# Crosstab local_experience × tender_year
crosstab = (
    pd.crosstab(universe["tender_year"], universe["local_experience"])
    .rename(columns={0: "no_experience", 1: "local_experience"})
    .reset_index()
    .sort_values("tender_year")
)

display(crosstab)

# Save output
crosstab_out = OUT_DIR / "local_experience_by_year.csv"
crosstab.to_csv(crosstab_out, index=False)

print("crosstab saved to:", crosstab_out.resolve())


local_experience,tender_year,no_experience,local_experience
0,2016,36,0
1,2017,19,5
2,2018,16,8
3,2019,12,0
4,2021,5,7


crosstab saved to: E:\Daniel\Stockholm\submission\notebooks\submission\stata\output\local_experience_by_year.csv
